In [1]:
from clipp2.core_cuda import *
from clipp2.preprocess import *
df1 = process_files('AVPC/ACC9/9-post.snv.txt', 'AVPC/ACC9/9-post.cna.txt','AVPC/ACC9/9-post.purity.txt')
df2 = process_files('AVPC/ACC9/9-pre.snv.txt', 'AVPC/ACC9/9-pre.cna.txt','AVPC/ACC9/9-pre.purity.txt')
[df1, df2] = insert_distinct_rows_multi([df1, df2])
export_snv_cna_and_purity(
        df1,
        dir="AVPC/ACC9pre/",
        snv_path="9post.snv.txt",
        cna_path="9post.cna.txt",
        purity_path="9post.purity.txt"
    )
export_snv_cna_and_purity(
        df2,
        dir="AVPC/ACC9pre/",
        snv_path="9pre.snv.txt",
        cna_path="9pre.cna.txt",
        purity_path="9pre.purity.txt"
    )
snv_file      = "AVPC/ACC9pre/9post.snv.txt"
cn_file       = "AVPC/ACC9pre/9post.cna.txt"
purity_file   = "AVPC/ACC9pre/9post.purity.txt"
sample_id     = "--sample_id"   # not heavily used in logic
output_prefix = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9post"

# Call function with drop_data=True (to replicate R code's dropping logic)
process_data(
    snv_file, cn_file, purity_file, sample_id, output_prefix, 
    drop_data=False
)
snv_file      = "AVPC/ACC9pre/9pre.snv.txt"
cn_file       = "AVPC/ACC9pre/9pre.cna.txt"
purity_file   = "AVPC/ACC9pre/9pre.purity.txt"
sample_id     = "--sample_id"   # not heavily used in logic
output_prefix = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9pre"

Loaded SNV data from AVPC/ACC9/9-post.snv.txt with shape (3862, 4)
Loaded CNA data from AVPC/ACC9/9-post.cna.txt with shape (181, 6)
Purity: 0.21
Constructed df with shape (3862, 11)
Loaded SNV data from AVPC/ACC9/9-pre.snv.txt with shape (5576, 4)
Loaded CNA data from AVPC/ACC9/9-pre.cna.txt with shape (277, 6)
Purity: 0.31
Constructed df with shape (5576, 11)
Process done. Created outputs in E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9post.


In [2]:
# Call function with drop_data=True (to replicate R code's dropping logic)
process_data(
    snv_file, cn_file, purity_file, sample_id, output_prefix, 
    drop_data=False
)
root_dir = "E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data"
(r, n, minor, total, purity, coef_list, wcut, drop) = group_all_regions_for_ADMM(root_dir)

Process done. Created outputs in E:/Dropbox/GitHub/Multi_Region_CliPP/processed_data/acc9pre.
Loaded region 'acc9post': r.shape=(5793,), coef.shape=(5793, 6), purity=0.21
Loaded region 'acc9pre': r.shape=(5793,), coef.shape=(5793, 6), purity=0.31

=== Summary of grouped data before dropping rows ===
Found M=2 regions. r shape= (5793, 2), n= (5793, 2)
minor= (5793, 2), total= (5793, 2)
coef_list length= 2 (each is (No_mutation,6))
wcut= [-0.18  1.8 ]

Dropped 0 rows that were all-zero in r/n/minor/total/coef.

=== Summary of grouped data after dropping rows ===
r shape= (5793, 2), n= (5793, 2)
minor= (5793, 2), total= (5793, 2)
coef_list length= 2, each => shape (5793, 6)


e:\Dropbox\GitHub\Multi_Region_CliPP\clipp2\preprocess.py:763: RuntimeWarning: divide by zero encountered in divide
  phi_arr = 2.0 / ( (minor_count / (minor_read/total_read)) - total_count + 2.0 )


In [3]:
wcut = [-1.8, 1.8]
alpha=0.8
gamma=3.7
rho=1.02
precision= 0.01
Run_limit=1e4
control_large=5
Lambda=0.1
post_th=0.05
least_diff=0.01
device = 'cuda'

In [4]:
def ensure_2D_and_no_zeros(arr):
    """
    - If arr is 1D, reshape to (No_mutation, 1).
    - If arr is 2D, keep shape.
    - Then replace any zeros with 1.
    """
    if arr.ndim == 1:
        arr = arr.reshape(-1, 1)
    elif arr.ndim != 2:
        raise ValueError(f"Expected 1D or 2D array, got shape {arr.shape}")
    # Replace zeros with 1
    # This is vectorized, no Python loops:
    arr = np.where(arr == 0, 1, arr)
    return arr

r     = ensure_2D_and_no_zeros(r)
n     = ensure_2D_and_no_zeros(n)
minor = ensure_2D_and_no_zeros(minor)
total = ensure_2D_and_no_zeros(total)

def to_torch_gpu(arr):
    return torch.as_tensor(arr, dtype=torch.float32, device=device)

r_t     = to_torch_gpu(r)
n_t     = to_torch_gpu(n)
minor_t = to_torch_gpu(minor)
total_t = to_torch_gpu(total)

# Build c_all => shape(No_mutation, M, 6)
c_stack = []
for c in coef_list:
    # Each c => shape(No_mutation, 6)
    c_stack.append(to_torch_gpu(c))
c_all_t = torch.stack(c_stack, dim=1)  # shape => (No_mutation, M, 6)

No_mutation, M = r_t.shape

# 1) Prepare purity => shape (No_mutation, M)
if isinstance(purity, (float, int)):
    purity_t = torch.full((No_mutation, M), float(purity), device=device)
else:
    purity_vec = to_torch_gpu(purity)  # shape (M,)
    purity_t   = purity_vec.unsqueeze(0).expand(No_mutation, -1)

ploidy_t = torch.full((No_mutation, M), 2.0, device=device)

# 2) Initialize w_new from logistic bounding
fraction_t = r_t / (n_t + 1e-12)
phi_hat_t  = fraction_t * ((ploidy_t - purity_t*ploidy_t) + (purity_t*total_t)) / (minor_t + 1e-12)

scale_parameter = torch.clamp(torch.max(phi_hat_t), min=1.0)
phi_new_t = phi_hat_t / scale_parameter

low_b = torch.sigmoid(torch.tensor(-control_large, device=device))
up_b  = torch.sigmoid(torch.tensor( control_large, device=device))
phi_new_t = torch.clamp(phi_new_t, low_b, up_b)

w_init_t = torch.log(phi_new_t) - torch.log(1 - phi_new_t + 1e-12)
w_init_t = torch.clamp(w_init_t, -control_large, control_large)
w_new_t  = w_init_t.clone()

# 3) Build the sparse DELTA operator (COO)
#    For i<j pairs, each row in the big (No_pairs*M) row-block has +1 at (i,m) and -1 at (j,m).
#    We'll do this *without* for-loops using advanced indexing.
i_idx_np, j_idx_np = torch.triu_indices(No_mutation, No_mutation, offset=1)
# Move them to 'device'
i_idx_np = i_idx_np.to(device)
j_idx_np = j_idx_np.to(device)

No_pairs = i_idx_np.size(0)

# Then proceed to build row/col indices for the sparse Delta operator:
pair_range = torch.arange(No_pairs, device=device)
m_range    = torch.arange(M, device=device)

# row_idx for the +1 block => shape (No_pairs, M)
row_plus_2D = pair_range.unsqueeze(1)*M + m_range  # broadcast
# row_idx for the -1 block => same row => row_plus_2D
row_combined = torch.cat([row_plus_2D, row_plus_2D], dim=1)  # shape => (No_pairs, 2*M)

# col_idx for +1 => i_idx_np*M + m
col_plus_2D = i_idx_np.unsqueeze(1)*M + m_range
# col_idx for -1 => j_idx_np*M + m
col_minus_2D = j_idx_np.unsqueeze(1)*M + m_range
col_combined = torch.cat([col_plus_2D, col_minus_2D], dim=1)

row_idx = row_combined.reshape(-1)
col_idx = col_combined.reshape(-1)

# data => +1 then -1
plus_block  = torch.ones_like(col_plus_2D, dtype=torch.float32)
minus_block = -torch.ones_like(col_minus_2D, dtype=torch.float32)
vals_2D = torch.cat([plus_block, minus_block], dim=1)
vals = vals_2D.reshape(-1)

total_rows = No_pairs*M
total_cols = No_mutation*M
Delta_coo = torch.sparse_coo_tensor(
    indices=torch.stack([row_idx, col_idx], dim=0),
    values=vals,
    size=(total_rows, total_cols),
    device=device
).coalesce()

# 4) Initialize eta, tau
#    We can get the difference directly: D_w => shape(No_pairs, M) = w[i_idx]-w[j_idx]
#    We'll re-use the code from scad_threshold_update for consistency.
#    For initialization, just do:
eta_new_t = (w_new_t[i_idx_np, :] - w_new_t[j_idx_np, :])
tau_new_t = torch.ones_like(eta_new_t, device=device)

# 5) ADMM iteration
residual = 1e6
k_iter   = 0

low_cut, up_cut = wcut

In [ ]:
while k_iter < Run_limit and residual > precision:

    # Might also stop if we get nan
    # (we do a small check below)
    k_iter += 1

    w_old_t  = w_new_t.clone()
    eta_old_t = eta_new_t.clone()
    tau_old_t = tau_new_t.clone()

    # =========== (A) IRLS expansions (vectorized) ===========
    expW_t = torch.exp(w_old_t)
    denom_ = 2.0 + expW_t*total_t
    denom_ = torch.clamp(denom_, min=1e-12)
    theta_t = (expW_t * minor_t)/denom_

    maskLow_t = (w_old_t <= low_cut)
    maskUp_t  = (w_old_t >= up_cut)
    maskMid_t = ~(maskLow_t | maskUp_t)

    # c_all_t => shape(No_mutation, M, 6)
    partA_full_t = (
        maskLow_t * c_all_t[...,1]
        + maskUp_t  * c_all_t[...,5]
        + maskMid_t * c_all_t[...,3]
    ) - (r_t/(n_t+1e-12))

    partB_full_t = (
        maskLow_t * c_all_t[...,0]
        + maskUp_t  * c_all_t[...,4]
        + maskMid_t * c_all_t[...,2]
    )

    sqrt_n_t = torch.sqrt(n_t)
    denom2_t = torch.sqrt(theta_t*(1-theta_t) + 1e-12)

    A_array_t = (sqrt_n_t * partA_full_t)/denom2_t
    B_array_t = (sqrt_n_t * partB_full_t)/denom2_t

    A_flat_t = A_array_t.flatten()  # shape => (No_mutation*M,)
    B_flat_t = B_array_t.flatten()

    # ---------------- (B) Build system & solve with manual CG ----------------

    # 1) Get dimensions and build diag(B^2):
    NM = B_flat_t.shape[0]      # = (No_mutation * M)
    B_sq_t = B_flat_t**2        # shape => (NM,)

    # 2) Compute sparse Delta^T Delta => shape (NM, NM)
    Delta_t = Delta_coo.transpose(0, 1)  # shape => (NM, No_pairs*M)
    DTD = torch.sparse.mm(Delta_t, Delta_coo)  # shape => (NM, NM), still sparse

    # 3) Define a function that multiplies H = (diag(B^2) + alpha * (Delta^T Delta)) by x
    def matvec_H(x):
        # x shape => (NM,)
        # (A) multiply by diag(B_sq_t)
        out = B_sq_t * x
        # (B) add alpha * (DTD @ x)
        out += alpha * torch.sparse.mm(DTD, x.unsqueeze(-1)).squeeze(-1)
        return out

    # 4) Build the right-hand side: linear_t
    big_eta_tau_t = alpha*eta_old_t + tau_old_t   # shape => (No_pairs, M)
    big_eta_tau_flat_t = big_eta_tau_t.flatten()  # shape => (No_pairs*M,)

    RHS_1 = torch.sparse.mm(Delta_t, big_eta_tau_flat_t.unsqueeze(1)).squeeze(1)
    linear_t = RHS_1 - (B_flat_t * A_flat_t)      # shape => (NM,)

    # 5) Manual Conjugate Gradient solve
    #    We'll do a 'while True' to avoid Python 'for' loops.
    #    x0: initial guess => use the old solution w_old_t.
    x = w_old_t.flatten().clone()
    r = linear_t - matvec_H(x)
    p = r.clone()
    rs_old = torch.dot(r, r)

    max_cg_iter = 500
    tol = 1e-6
    iter_cg = 0

    while True:
        # Ap = H * p
        Ap = matvec_H(p)
        denom_ = torch.dot(p, Ap) + 1e-12
        alpha_cg = rs_old / denom_
        
        x = x + alpha_cg * p
        r = r - alpha_cg * Ap
        rs_new = torch.dot(r, r)

        iter_cg += 1
        # Check stopping criteria
        if rs_new.sqrt() < tol or iter_cg >= max_cg_iter:
            break

        p = r + (rs_new / rs_old) * p
        rs_old = rs_new

    # x now holds the solution for (diag(B^2) + alpha * DTD) * x = linear_t
    w_new_flat_t = x
    w_new_t = w_new_flat_t.view(No_mutation, M)

    # Finally, clamp in [-control_large, control_large]
    w_new_t = torch.clamp(w_new_t, -control_large, control_large)


    # Finally, clamp in [-control_large, control_large]
    w_new_t = torch.clamp(w_new_t, -control_large, control_large)

    # =========== (C) SCAD threshold ===========  
    eta_new_t, tau_new_t = scad_threshold_update_torch(
        w_new_t, tau_old_t, Delta_coo, alpha, Lambda, gamma
    )

    # scale alpha
    alpha *= rho

    # =========== (D) residual check ===========
    # residual = max_{i<j,m} | (w[i]-w[j]) - eta[k]| 
    # We'll do it again with the Delta approach
    # D_w = Delta_coo w_new_flat
    w_new_flat2 = w_new_t.flatten()
    D_w_flat2   = torch.sparse.mm(Delta_coo, w_new_flat2.unsqueeze(1)).squeeze(1)
    # reshape to (No_pairs, M)
    D_w2 = D_w_flat2.view(No_pairs, M)
    diff_2D_t = D_w2 - eta_new_t
    residual_val_t = torch.max(torch.abs(diff_2D_t))
    residual = float(residual_val_t.item())

    if torch.isnan(residual_val_t):
        break

    print(f"Iter={k_iter}, alpha={alpha:.4f}, residual={residual:.6g}")

print("\nADMM finished.\n")

Iter=1, alpha=0.8160, residual=1.375


In [ ]:
w_new = w_new_t.detach().cpu().numpy()
eta_new = eta_new_t.detach().cpu().numpy()
phi_hat = phi_hat_t.detach().cpu().numpy()
diff = diff_mat(w_new)
ids = np.triu_indices(diff.shape[1], 1)
eta_new[np.where(np.abs(eta_new) <= post_th)] = 0
diff[ids] = np.linalg.norm(eta_new, axis=1)
class_label = -np.ones(No_mutation)
class_label[0] = 0
group_size = [1]
labl = 1

for i in range(1, No_mutation):
    for j in range(i):
        if diff[j, i] == 0:
            class_label[i] = class_label[j]
            group_size[int(class_label[j])] += 1
            break
    if class_label[i] == -1:
        class_label[i] = labl
        labl += 1
        group_size.append(1)

# quality control
least_mut = np.ceil(0.05 * No_mutation)
tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
tmp_grp = np.where(group_size == tmp_size)
refine = False
if tmp_size < least_mut:
    refine = True

while refine:
    refine = False
    tmp_col = np.where(class_label == tmp_grp[0][0])[0]
    for i in range(len(tmp_col)):
        if tmp_col[i] != 0 and tmp_col[i] != No_mutation - 1:
            tmp_diff = np.abs(np.append(np.append(diff[0:tmp_col[i], tmp_col[i]].T.ravel(), 100),
                                        diff[tmp_col[i], (tmp_col[i] + 1):No_mutation].ravel()))
            tmp_diff[tmp_col] += 100
            diff[0:tmp_col[i], tmp_col[i]] = tmp_diff[0:tmp_col[i]]
            diff[tmp_col[i], (tmp_col[i] + 1):No_mutation] = tmp_diff[(tmp_col[i] + 1):No_mutation]
        elif tmp_col[i] == 0:
            tmp_diff = np.append(100, diff[0, 1:No_mutation])
            tmp_diff[tmp_col] += 100
            diff[0, 1:No_mutation] = tmp_diff[1:No_mutation]
        else:
            tmp_diff = np.append(diff[0:(No_mutation - 1), No_mutation - 1], 100)
            tmp_diff[tmp_col] += 100
            diff[0:(No_mutation - 1), No_mutation - 1] = tmp_diff[0:(No_mutation - 1)]
        ind = tmp_diff.argmin()
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] -= 1
        class_label[tmp_col[i]] = class_label[ind]
        group_size[class_label.astype(np.int64, copy=False)[tmp_col[i]]] += 1
    tmp_size = np.min(np.array(group_size)[np.array(group_size) > 0])
    tmp_grp = np.where(group_size == tmp_size)
    refine = False
    if tmp_size < least_mut:
        refine = True

labels = np.unique(class_label)
phi_out = np.zeros((len(labels), M))
for i in range(len(labels)):
    ind = np.where(class_label == labels[i])[0]
    class_label[ind] = i
    phi_out[i, :] = np.sum(phi_hat[ind, : ] * n[ind, : ], axis=0) / np.sum(n[ind, : ], axis=0)

if len(labels) > 1:
    sort_phi = sort_by_2norm(phi_out)
    phi_diff = sort_phi[1:, :] - sort_phi[:-1, :]
    min_ind, min_val = find_min_row_by_2norm(phi_diff)
    while np.linalg.norm(min_val) < least_diff:
        combine_ind = np.where(phi_out == sort_phi[min_ind, ])[0]
        combine_to_ind = np.where(phi_out == sort_phi[min_ind + 1, ])[0]
        class_label[class_label == combine_ind] = combine_to_ind
        labels = np.unique(class_label)
        phi_out = np.zeros((len(labels), M))
        for i in range(len(labels)):
            ind = np.where(class_label == labels[i])[0]
            class_label[ind] = i
            phi_out[i] = np.sum(phi_hat[ind, : ] * n[ind, : ], axis=0) / np.sum(n[ind, : ], axis=0)
        if len(labels) == 1:
            break
        else:
            sort_phi = sort_by_2norm(phi_out)
            phi_diff = sort_phi[1:, :] - sort_phi[:-1, :]
            min_ind, min_val = find_min_row_by_2norm(phi_diff)
phi_res = np.zeros((No_mutation, M))
for lab in range(np.shape(phi_out)[0]):
    phi_res[class_label == lab, ] = phi_out[lab, ]

NameError: name 'w_new_t' is not defined